In [2]:

!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 974.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=807e05b7b490f0a081baa05e9b0eaddaa4fc5dfa3dbb2c50431c4bb02e060086
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=1925a2085d47e299f472eeb5833b492ee567b04cb1d984b25df0f33d41a3690a
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built e

In [47]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [48]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [55]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 7)
    layers = []
    in_channels = [3]
    out_size = 224
    for i in range(0, n_layers):
        out_channels = trial.suggest_int(f"{i} layer output channel", in_channels[i]+1, in_channels[i]+20)
        in_channels.append(out_channels)
        layers.append(nn.Conv2d(in_channels=in_channels[i], out_channels=out_channels, kernel_size=3))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
        out_size = out_size - 3 + 1                                       # Size of the output kernel
        out_size = int(out_size/2) 
    layers.append(nn.Flatten())
    layers.append(nn.Linear(in_channels[-1]*out_size*out_size, 4))
    layers.append(nn.LogSoftmax(dim=1))
    return nn.Sequential(*layers)


In [56]:
def get_data():
    train_path = ('/kaggle/input/oasis-brain/TRAIN/TRAIN/')
    test_path = ('/kaggle/input/oasis-brain/TEST/TEST/test/')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root=train_path, transform=transform)
    test_dataset = ImageFolder(root=test_path, transform=transform)
    train_loader=DataLoader(
        train_dataset,
        batch_size=32, shuffle=True,
    )
    test_loader=DataLoader(
        test_dataset,
        batch_size=32, shuffle=True,
    )
    return train_loader, test_loader

In [57]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    model.train()
    correct = 0
    total = 0

    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            if i >= 20:
                break
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}, Train Accuracy: {accuracy:.2f}%")

In [58]:
def test(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        #for inputs, labels in test_loader:
        for i, (inputs, labels) in enumerate(test_loader):
            if i >= 10:
                break
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [59]:
def objective(trial):
    model = define_model(trial).to(device)
    train_dataloader, val_dataloader = get_data()
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    train(model, train_loader, 1, learning_rate, device)
    accuracy = test(model, test_loader, device)
    trial.report(accuracy, 0)
    if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return accuracy
    

In [60]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=75, timeout=6000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-04 17:35:52,257] A new study created in memory with name: no-name-a7faf4c1-ce96-457c-8ac3-6991754b0a01


Epoch 1/1, Loss: 0.00574679840776257, Train Accuracy: 78.59%


[I 2024-04-04 17:36:45,035] Trial 0 finished with value: 59.375 and parameters: {'n_layers': 4, '0 layer output channel': 10, '1 layer output channel': 19, '2 layer output channel': 29, '3 layer output channel': 34, 'lr': 0.0003545526217073593}. Best is trial 0 with value: 59.375.


Test Accuracy: 59.38%
Epoch 1/1, Loss: 0.012426226858108763, Train Accuracy: 20.31%


[I 2024-04-04 17:36:56,348] Trial 1 finished with value: 29.0625 and parameters: {'n_layers': 5, '0 layer output channel': 9, '1 layer output channel': 15, '2 layer output channel': 18, '3 layer output channel': 38, '4 layer output channel': 56, 'lr': 1.1553724367556949e-05}. Best is trial 0 with value: 59.375.


...

Test Accuracy: 57.19%
Epoch 1/1, Loss: 0.017115344442022572, Train Accuracy: 72.03%


[I 2024-04-04 17:40:05,048] Trial 6 pruned. 


Test Accuracy: 59.06%


/opt/conda/lib/python3.10/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
[W 2024-04-04 17:40:13,124] Trial 7 failed with parameters: {'n_layers': 7, '0 layer output channel': 18, '1 layer output channel': 25, '2 layer output channel': 37, '3 layer output channel': 44, '4 layer output channel': 52, '5 layer output channel': 55, '6 layer output channel': 73, 'lr': 8.814457430201423e-05} because of the following error: RuntimeError("Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size").
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_34/73075519.py", line 5, in objective
    train(model, train_loader, 1, learning_rate, device)
  File "/tmp/ipykernel_34/3200141229.py"

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [46]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=75, timeout=6000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-04 17:14:02,589] A new study created in memory with name: no-name-d2351ffb-7129-4430-b74c-940408f510fa


Epoch 1/1, Loss: 0.006913617253303528, Train Accuracy: 73.59%


[I 2024-04-04 17:15:29,579] Trial 0 finished with value: 31.25 and parameters: {'n_layers': 5, '0 layer output channel': 13, '1 layer output channel': 24, '2 layer output channel': 32, '3 layer output channel': 42, '4 layer output channel': 55, 'lr': 0.00016430845883314251}. Best is trial 0 with value: 31.25.


Test Accuracy: 31.25%
Epoch 1/1, Loss: 0.006560513773060084, Train Accuracy: 75.94%


[I 2024-04-04 17:16:23,846] Trial 1 finished with value: 59.375 and parameters: {'n_layers': 5, '0 layer output channel': 18, '1 layer output channel': 19, '2 layer output channel': 31, '3 layer output channel': 34, '4 layer output channel': 49, 'lr': 0.00027176721829712665}. Best is trial 1 with value: 59.375.


...

Test Accuracy: 56.88%
Epoch 1/1, Loss: 0.006623161429151855, Train Accuracy: 76.72%


[I 2024-04-04 17:33:07,935] Trial 28 pruned. 


Test Accuracy: 59.69%


[W 2024-04-04 17:33:23,287] Trial 29 failed with parameters: {'n_layers': 5, '0 layer output channel': 13, '1 layer output channel': 26, '2 layer output channel': 38, '3 layer output channel': 52, '4 layer output channel': 68} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_34/73075519.py", line 3, in objective
    train_dataloader, val_dataloader = get_data()
  File "/tmp/ipykernel_34/2514366985.py", line 9, in get_data
    train_dataset = ImageFolder(root=train_path, transform=transform)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/datasets/folder.py", line 309, in __init__
    super().__init__(
  File "/opt/conda/lib/python3.10/site-packages/torchvision/datasets/folder.py", line 145, in __init__
    samples = self.make_dataset(self.root, class_to_idx, extensions, is_valid_fi

KeyboardInterrupt: 

Trial 2 finished with value: 63.4375 and parameters: {'n_layers': 5, '0 layer output channel': 16, '1 layer output channel': 29, '2 layer output channel': 36, '3 layer output channel': 45, '4 layer output channel': 64, 'lr': 5.973592490223275e-05}. Best is trial 2 with value: 63.4375.

In [84]:
class ConvNet_1(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet_1, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=36, kernel_size=3),
            nn.BatchNorm2d(36),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=36, out_channels=45, kernel_size=3),
            nn.BatchNorm2d(45),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=45, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc = nn.Linear(64*12*12, num_classes)
        #self.lsm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        #out = self.lsm(out)
        return out

Trial 7 finished with value: 62.1875 and parameters: {'n_layers': 3, '0 layer output channel': 22, '1 layer output channel': 34, '2 layer output channel': 48, 'lr': 0.003775760161262146}. Best is trial 7 with value: 62.1875.

In [87]:
class ConvNet_2(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet_2, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=22, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=34, kernel_size=3),
            nn.BatchNorm2d(34),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=34, out_channels=48, kernel_size=3),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.fc = nn.Linear(48*26*26, num_classes)
        #self.lsm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        #out = self.lsm(out)
        return out